# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# nlp libraries
import nltk
# nltk.download() # run only once

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [22]:
# scikit-learn libraries
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report

In [15]:
#in case that autocomplete does not work on the notebook
%config Completer.use_jedi = False

In [6]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('messages', engine)

X = df['message'].values # english-translated message
Y = df.iloc[:, -36:]

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    # Lowercase the text
    # Make a list of tokenized words (trimmed) under the conditions that
    #    1. word is not stopword
    #    2. word is alpha-numumeric, excluding (space)!#%&?, etc
    tokens = [
        word.strip() for word in word_tokenize(text.lower())
        if (word not in stopwords.words('english') and  # not stopword
            word.isalnum()) # is alpha-numeric word           
    ]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer() # instantiate a lemmatizer
    lemmed = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmed

In [8]:
# Testing the function 
sample_texts = X[:5]

for text in sample_texts :
    print(f'Original text  : {text}')
    print(f'Tokenized text : {tokenize(text)}')
    print()

Original text  : Weather update - a cold front from Cuba that could pass over Haiti
Tokenized text : ['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

Original text  : Is the Hurricane over or is it not over
Tokenized text : ['hurricane']

Original text  : Looking for someone but no name
Tokenized text : ['looking', 'someone', 'name']

Original text  : UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
Tokenized text : ['un', 'report', 'leogane', 'destroyed', 'hospital', 'croix', 'functioning', 'need', 'supply', 'desperately']

Original text  : says: west side of Haiti, rest of the country today and tonight
Tokenized text : ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
clf_pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline

In [19]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
# Predict on X_test
Y_pred = clf_pipeline.predict(X_test)

In [64]:
# The below code returns error
# print(classification_report(Y_test.values, Y_pred, target_names=Y.columns.values))

`Y_test` is a dataframe and `Y_pred` is an array -- need data type convserion <br>
`classification_report` needs inputs in 1-d array. So iterate by column and print the result

In [49]:
# Coverting Y_pred into a dataframe
Y_pred_df = pd.DataFrame(Y_pred.values, columns=Y_test.columns.values)
Y_pred_df.head(3) # 36 features

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
# Printing classfication report
# for col in Y.columns.values : 
#     print(f'Classification score for Category {col}', end='\n\n')
#     print(classification_report(Y_test[col], Y_pred_df[col]))
#     print()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.